## Ising model

In [ ]:
import matplotlib.pyplot as plt
import sys
from numba import njit  
import numpy as np  
import random  
import time  
from tqdm import tqdm  
from math import log
from collections import Counter

In [ ]:
#Simulation parameters
B = 0  # Magnetic field strength
L = 20  # Lattice size (width)
n = 1000 * L ** 2  # number of MC sweeps
Temperature = np.arange(1.0,4.1,0.1) 

In [ ]:
# Calculate interaction energy between spins. Assume periodic boundary conditions
# Interaction energy will be the difference in energy due to flipping spin i,j
@njit  
def dE(s, i, j):  # change in energy function
    # top
    if i == 0:
        t = s[L - 1, j]  
    else:
        t = s[i - 1, j]
    # bottom
    if i == L - 1:
        b = s[0, j]  
    else:
        b = s[i + 1, j]
    # left
    if j == 0:
        l = s[i, L - 1] 
    else:
        l = s[i, j - 1]
    # right
    if j == L - 1:
        r = s[i, 0]  
    else:
        r = s[i, j + 1]
    return 2 * s[i, j] * (t + b + r + l)  


# Monte-carlo sweep implementation
@njit  
def mc(s, Temp, n):
    for m in range(n):
        i = random.randrange(L)  # choose random row
        j = random.randrange(L)  # choose random column
        ediff = dE(s, i, j)
        if ediff <= 0:  # if the change in energy is negative
            s[i, j] = -s[i, j]  # accept move and flip spin
        elif random.random() < np.exp(-ediff / Temp):  # if not accept it with probability exp^{-dU/kT}
            s[i, j] = -s[i, j]
    return s


def entropy(frequencies, base=2):
    return -1 * sum([f/sum(frequencies) * log(f/sum(frequencies), base) for f in frequencies])



In [ ]:
# Simulate at particular temperatures (T) and compute physical quantities (Energy, heat capacity and magnetization)
entropies=[]
for  T in tqdm(Temperature):
    #frequencies = Counter() --> for low dimension, entropy by enumeration
    file_name = 'configurations_{temperature}'.format(temperature = str(round(T, 2)))
    for ind in range(20000):
      s = np.random.choice([1, -1], size=(L, L))
      # Sweeps spins
      s = mc(s, T, n)
      lattice = np.array(s)
      #frequencies[str(lattice)] += 1 --> for low dimension, entropy by enumeration
      if ind==0:
        lattices=np.array([lattice])
      else:
        lattices= np.concatenate((lattices, np.array([lattice])), axis=0)


    #entropies.append(entropy(frequencies.values())/(L**2)) --> for low dimension, entropy by enumeration
    np.save(file_name+'.npy', lattices)

    #Since there are just two spin configuration (+1,-1), to save storing space:
    ##lattices[lattices==-1.0]=0
    #np.savez_compressed(file_name+'.npz', np.packbits(lattices, axis=-1))


#print(f'Entropies : {entropies}')
#plt.plot(Temperature, np.array(entropies)) --> for low dimension, entropy by enumeration

## XY model

In [ ]:
#matplotlib inline
from __future__ import division
import numpy as np
import random as rand
from numpy import pi as pi
from numpy import cos as cos
from numpy import sin as sin
from numpy import exp as exp
from numpy import mod as mod
from numpy import absolute as absolute
from numba import jit
from numba import njit
##matplotlib inline
from __future__ import division, print_function
import numpy as np
from numpy.random import rand
import time
import sys
import os
#import zipfile
#from memory_profiler import profile
from joblib import Parallel, delayed
import math
from numpy import pi as pi
from numpy import cos as cos
from numpy import sin as sin
from numpy import exp as exp
from numpy import mod as mod
from numpy.random import randint as randint
from numpy import absolute as absolute
#from tqdm import tqdm
from tqdm.notebook import tqdm

In [ ]:
#one step of the Wolff
@njit
def WolffUpdate(config, temp, N, neighbors_list):
    beta=1./temp

    numItTot = N*N
    size = N*N
    #initialize outputs
    avg_size_clust = 0.

    cluster = np.zeros(size, dtype = np.int8)
    listQ = np.zeros(size + 1, dtype = np.int64)

    for nn in range(numItTot):
    
        init = np.random.randint(0, size)
        listQ[0] = init + 1
        theta_rand = (np.pi)*np.random.rand()   
        random_angle =  theta_rand

        cluster[init] = 1 #this site is in the cluster now
        sc_in = 0
        sc_to_add = 1

        while listQ[sc_in] != 0:
            site_studied = listQ[sc_in] + (-1)
            sc_in += 1
            avg_size_clust += 1

            prime_layer_rand = random_angle
            site_angle = config[site_studied]  #find the current angle of the studied site
            config[site_studied] = (2*prime_layer_rand - site_angle) #site selected has been flipped

            for kk in range(4):
                site_nn = neighbors_list[4*site_studied + kk]
                near_angle = config[site_nn]
                if cluster[site_nn] == 0:
                    energy_difference = (-1)*(cos(site_angle - near_angle) - cos(site_angle - (2*prime_layer_rand - near_angle)))
                    freezProb_next = 1. - exp(beta*energy_difference)
                    if (np.random.rand() < freezProb_next):
                        #listQ.append(site_nn)
                        listQ[sc_to_add] = site_nn + (1)
                        cluster[site_nn] = 1
                        sc_to_add += 1
        listQ[:] = 0
        cluster[:] = 0

    #average size cluster
    avg_size_clust = avg_size_clust/numItTot

    return avg_size_clust


@njit
def PTTstepTherm(config_init, temp, N, neighbors_list, factorIter):
  #the config
  config = config_init.copy()

  for st in range(factorIter):
    WolffUpdate(config, temp, N, neighbors_list)

    #final energy
    #energy = energies[-1]

  return config

In [ ]:
N = 20 #int(sys.argv[1])  #note that N is in fact L the linear size
factor_print = 10000
jint = 1.0

L_size = N

#temp range
Tmin = 0.1
Tmax = 1.5
nt = 32
type_of_temp_range = 1

#number of steps
num_cores = 1
length_box = 100 
therm = 500

Beta_min = 1/Tmax
Beta_max = 1/Tmin

#the list of temperatures and the list of energies, initialized
if type_of_temp_range == 0:
  ratio_T = (Tmax/Tmin)**(1/(nt-1))
  range_temp = np.zeros(nt)
  for i in range(nt):
    range_temp[i]=Tmin*((ratio_T)**(i))
elif type_of_temp_range == 1:
    range_temp = np.linspace(Tmin, Tmax, nt)
range_temp = [0.4]

list_temps = range_temp
list_energies = np.zeros(nt)

#initialize list of neighbors
neighbors_list = np.zeros(4*(N**2))
sizeN = N**2
for i in range(N**2):
  vec_nn_x = [-1, 1, 0, 0]
  vec_nn_y = [0,  0, -1,1]

  site_studied_1 = i//N
  site_studied_2 = i%N
  for p in range(4):
    neighbors_list[4*i + p] = (N*mod((site_studied_1 + vec_nn_x[p]),N) + mod((site_studied_2 + vec_nn_y[p]),N))
        #neighbors_list[5*i + 4] = ((i + sizeN)%(2*sizeN))

neighbors_list = np.array(list(map(int, neighbors_list)))

for  T in tqdm(range_temp):
    file_name = 'configurations_{temperature}'.format(temperature = str(round(T, 2)))
    for ind in range(1):
      s = 2*pi*rand(N**2)
      # Sweeps spins
      s = PTTstepTherm(config_init = s,temp = T, N = N, neighbors_list = neighbors_list, factorIter = therm)
      lattice = np.array(s).reshape(N,N)
      if ind==0:
        lattices=np.array([lattice])
      else:
        lattices= np.concatenate((lattices, np.array([lattice])), axis=0)
    np.save(file_name+'.npy', lattices)


## Spin Glass

In [ ]:
import matplotlib.pyplot as plt
import sys
from numba import njit  
import numpy as np  
import random  
import time  
from tqdm import tqdm  
from math import log
import torch

In [ ]:
@njit
def dE(s, i, j, up, down, left, right):  # change in energy function
    # top
    if i == 0:
        t = s[L - 1, j]  # periodic boundary (top)
    else:
        t = s[i - 1, j]
    # bottom
    if i == L - 1:
        b = s[0, j]  # periodic boundary (bottom)
    else:
        b = s[i + 1, j]
    # left
    if j == 0:
        l = s[i, L - 1]  # periodic boundary (left)
    else:
        l = s[i, j - 1]
    # right
    if j == L - 1:
        r = s[i, 0]  # periodic boundary  (right)
    else:
        r = s[i, j + 1]
    return 2 * s[i, j] * (t*up[i,j] + b*down[i,j] + r*right[i,j] + l*left[i,j])  # difference in energy is i,j is flipped


# Monte-carlo sweep implementation
@njit  
def mc(s, Temp, n, up, down, left, right):
    for m in range(n):
        i = random.randrange(L)  # choose random row
        j = random.randrange(L)  # choose random column
        ediff = dE(s, i, j , up, down, left,right)
        if ediff <= 0:  # if the change in energy is negative
            s[i, j] = -s[i, j]  # accept move and flip spin
        elif random.random() < np.exp(-ediff / Temp):  # if not accept it with probability exp^{-dU/kT}
            s[i, j] = -s[i, j]
    return s

In [ ]:
up = np.zeros((L,L))
down = np.zeros((L,L))
left = np.zeros((L,L))
right = np.zeros((L,L))

s_h = 1
s_v = 1
up[:,:] = np.random.normal(size=(L,L)) * s_v
down[0:L-1,:] = up[1:L,:]
down[L-1,:]=up[0,:]
left[:,:] = np.random.normal(size=(L,L)) * s_h
right[:,0:L-1] = left[:,1:L]
right[:,L-1] = left[:,0]


# Simulate at particular temperatures (T) and compute physical quantities (Energy, heat capacity and magnetization)
for  T in tqdm(Temperature):
    print(T)
    file_name = 'configurations_{temperature}'.format(temperature = str(round(T, 4)))
    for ind in range(1000):
      s = np.random.choice([1, -1], size=(L, L))

      # Sweeps spins
      s = mc(s, T, n, up, down, left, right)
      lattice = np.array(s)
      if ind==0:
        lattices=np.array([lattice])
      else:
        lattices= np.concatenate((lattices, np.array([lattice])), axis=0)
    
    np.save(file_name+'.npy', lattices)
    

In [ ]:
#some useful plots to visualize configurations at different temperatures
i=0
N=10
fig, axs = plt.subplots(len(Temperature),N, figsize=(10,10))
for t in Temperature:
  Xb = np.load('./configurations_{temperature}.npy'.format(temperature=str(round(t, 4))))
  X_tensor = torch.Tensor(Xb)

  for j, index in enumerate(np.random.randint(0,1000,(N))):
    axs[i,j].imshow(X_tensor[index], interpolation='none')
  i+=1